In [63]:
def car(env, name, bcs, driving_time, charge_duration):
    
    yield env.timeout(driving_time)
    
    #request charging spot
    print('%s arriving at %d' %(name, env.now))
    
    with bcs.request() as req:
        yield req
        
        print('%s starting to charge at %s' %(name, env.now))
        yield  env.timeout(charge_duration)
        
        print('%s leaving the bcs at %s' %(name, env.now))

In [64]:
import simpy
env = simpy.Environment()
bcs = simpy.Resource(env, capacity = 2)

for i in range(4):
    env.process(car(env,'Car %d' %i, bcs, i*2, 5))

env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


In [31]:
env.process(car(env))

<Process(car) object at 0x107b107f0>

In [32]:
env.run(until=25)

start parking at 0
start driving at 5
start parking at 7
start driving at 12
start parking at 14
start driving at 19
start parking at 21


In [58]:
class Car(object):
    def __init__(self, env):
        self.env=env
        #start run process
        self.action = env.process(self.run())
    
    def run(self):
        while True:
            print('Start parking and charging at %d' %self.env.now)
            charge_duration = 5
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                print('Interrupted charging')
            
            #charge is done, we can start driving
            print('start driving at %d' % self.env.now)
            trip_duration = 2;
            yield self.env.timeout(trip_duration)
        
    def charge(self,duration):
        yield self.env.timeout(duration)
        
def driver(env,car):
    yield env.timeout(3)
    car.action.interrupt()

In [60]:
import simpy
env=simpy.Environment()
car = Car(env)
env.process(driver(env,car))
env.run(until=15)

Start parking and charging at 0
Interrupted charging
start driving at 3
Start parking and charging at 5
start driving at 10
Start parking and charging at 12


In [52]:
env.run(until=30)